In [1]:
import numpy as np
import pandas as pd
import duckdb
# suppress scientific notation by setting float_format
pd.set_option('display.float_format', '{:f}'.format)

In [2]:
# Conect to Database
con = duckdb.connect(database='database.db', read_only=False)
con.execute('INSTALL spatial')
con.execute('LOAD spatial')
#TODO build training data
con.sql('SHOW TABLES').fetchdf()

,name
0,compustat
1,compustat_geocoded
2,fnam
3,fnpm
4,matched
5,nets_all
6,nets_pub


In [3]:
con.sql('SELECT COUNT (*) FROM compustat').fetchdf()

,count_star()
0,39782


In [4]:
con.execute('SELECT * FROM compustat LIMIT 1000').fetchdf().sample(10)

,gvkey,hqcompany,hqzipcode,city,lat,lon
484,5133,BONTEXINC,24416,BUENA VISTA,37.744292,-79.365747
518,5386,GULFUSACORP,83837,KELLOGG,47.544760,-116.134760
187,2729,CAPITALCITIES/ABCINC,10023,NEW YORK,40.632098,-74.013308
414,4488,EXETEROILCOLTD-CLA,,,60.108670,-113.642580
192,2755,CARDIOSEARCHINC,33607,TAMPA,27.960150,-82.512272
479,5107,GENERALWATERWORKSCORP,19804-0508,WILMINGTON,39.737610,-75.532980
361,4092,DUKE&COINC,1002-24731,NEW YORK,40.758544,-73.968458
390,4294,ELECTRONICTRANSISTORSCORP,10550,HEMPSTEAD,40.704301,-73.637071
417,4516,FPINDINCOLD,96813,HONOLULU,21.307781,-157.863508
808,7804,NAVAJOFREIGHTLINESINC,,,NaN,NaN


In [5]:
# DuckDB configuration
# con.execute('SET memory_limit="10GB"')
# con.execute('SET threads TO 1')

In [6]:
# Get one random row of nets
con.sql('SELECT * FROM nets_pub ORDER BY RANDOM() LIMIT 1').fetchdf()

,hqduns,hqcompany,latitude,longitude
0,878115161,CSGSYSTEMSINTL,39.924500,-104.977600


In [7]:
prepared_statement_name_pub = """
    CREATE TABLE tmp AS
    SELECT
        hqduns, 
        hqcompany,
        latitude,
        longitude,
        jaro_winkler_similarity(n.hqcompany, ?) AS similarity_score
    FROM
        nets_pub n
        --we can use other similarity functions
        --we can also use other similarity scores
"""
prepared_statement_name_all = """
    CREATE TABLE tmp AS
    SELECT
        hqduns, 
        hqcompany,
        latitude,
        longitude,
        jaro_winkler_similarity(n.hqcompany, ?) AS similarity_score
    FROM
        nets_all n
        WHERE hqcity = ?
        --we can use other similarity functions
        --we can also use other similarity scores
"""

prepared_statement_add_match = """
    INSERT INTO matched
    VALUES (?, --gvkey 
            (SELECT hqcompany FROM compustat WHERE gvkey = ?), --hqcompany
            ?, --hqduns_pub
            (SELECT hqcompany FROM nets_pub WHERE hqduns = ?),
            ?, --name similarity_pub
            ?, --distance pub
            ?, --hqduns_all
            (SELECT hqcompany FROM nets_all WHERE hqduns = ?),
            ?,  --name similarity_all
            ?); --distance all
"""

prepared_statement_geo_match_all = """
    SELECT 
        hqduns,
        similarity_score,
        ST_Distance(
            ST_Point(n.latitude, n.longitude)::GEOMETRY,
            ST_Point(?, ?)::GEOMETRY
            ) AS distance
    FROM fnam n
    ORDER BY distance ASC
    LIMIT 1;
"""

prepared_statement_geo_match_pub = """
    SELECT 
        hqduns,
        similarity_score,
        ST_Distance(
            ST_Point(n.latitude, n.longitude)::GEOMETRY,
            ST_Point(?, ?)::GEOMETRY
            ) AS distance
    FROM fnpm n
    ORDER BY distance ASC
    LIMIT 1;
"""

In [8]:
# Get the matches for a specific name
def name_similarity_pub(comp_name):
    con.execute('DROP TABLE IF EXISTS tmp;')
    con.execute(prepared_statement_name_pub, [comp_name])
    return con.sql('SELECT * FROM tmp WHERE similarity_score > 0.9 ORDER BY similarity_score DESC LIMIT 5').fetchdf() #TODO change threshold

def name_similarity_all(comp_name, city):
    con.execute('DROP TABLE IF EXISTS tmp;')
    con.execute(prepared_statement_name_all, [comp_name, city])
    return con.sql('SELECT * FROM tmp WHERE similarity_score > 0.9 ORDER BY similarity_score DESC LIMIT 10').fetchdf() #TODO change threshold


In [9]:
#con.execute('DROP TABLE IF EXISTS matched;')
sql_query = """
CREATE TABLE IF NOT EXISTS matched (
    gvkey INTEGER,
    hqcompany TEXT,
    hqduns_pub INTEGER,
    hqcompany_pub TEXT,
    similarity_pub FLOAT,
    distance_pub FLOAT,
    hqduns_all INTEGER,
    hqcompany_all TEXT,
    similarity_all FLOAT,
    distance_all FLOAT
);
"""
con.execute(sql_query)

In [10]:

compustat_df = con.sql('SELECT * FROM compustat').fetchdf()

# Fill missing values
compustat_df['lat'] = compustat_df['lat'].fillna(0)
compustat_df['lon'] = compustat_df['lon'].fillna(0)
# compustat_df['phone'] = compustat_df['phone'].fillna('')
#compustat_df = compustat_df[0:2000]

In [11]:
# Stats Counter
pub_match_all = 0

In [12]:
def matching(compustat_df):
    pub_match_all = 0
    i = 0
    for index, row in compustat_df.iterrows():
        i += 1
        if i % 50 == 0:
            print(f'Processing {i} of {compustat_df.shape[0]}')
        #TODO get rid of this print
        gvkey = row['gvkey']
        #check if gvkey is already in matched
        if con.sql(f'SELECT COUNT(*) FROM matched WHERE gvkey = {gvkey}').fetchdf()['count_star()'].iloc[0] > 0:
            print(f'Already processed {gvkey}, {row["hqcompany"]}')
            continue
        # Get candidate matches
        name_pub_match = name_similarity_pub(row['hqcompany'])
        
        try:
            filtered_name_pub_matches = name_pub_match[name_pub_match['similarity_score'] == name_pub_match['similarity_score'].iloc[0]]
            # Create a temporary table with the filtered results
            con.execute('DROP TABLE IF EXISTS fnpm;')
            con.execute('DROP TABLE IF EXISTS tmp;')
            con.execute('CREATE TABLE fnpm AS SELECT * FROM filtered_name_pub_matches;')
            pub_match = con.execute(prepared_statement_geo_match_pub, [row['lat'], row['lon']]).fetchdf().iloc[0]
        except IndexError:
            pub_match = pd.DataFrame(columns= ['hqduns', 'similarity_score', 'distance'])
            pub_match.loc[0] = [-1, -1, 9999999]  # Fill with default values
            
        name_all_matches = name_similarity_all(row['hqcompany'], row['city'])
        
        try:
            # Get the top similarity score
            top_similarity_score = name_all_matches['similarity_score'].iloc[0]
            # Filter the DataFrame
            filtered_name_all_matches = name_all_matches[name_all_matches['similarity_score'] == top_similarity_score]
            # Create a temporary table with the filtered results
            con.execute('DROP TABLE IF EXISTS fnam;')
            con.execute('DROP TABLE IF EXISTS tmp;')
            con.execute('CREATE TABLE fnam AS SELECT * FROM filtered_name_all_matches;')
            name_all_match = con.execute(prepared_statement_geo_match_all, [row['lat'], row['lon']]).fetchdf().iloc[0]
        except IndexError:
            name_all_match = pd.DataFrame(columns= ['hqduns', 'similarity_score', 'distance'])
            name_all_match.loc[0] = [-1, -1, 9999999]  # Fill with default values
    
        if pub_match['hqduns'].item() in name_all_matches['hqduns'].values and pub_match['similarity_score'].item() >= name_all_match['similarity_score'].item():
            name_all_match = pub_match
            pub_match_all += 1
        
        try:    
            con.execute(prepared_statement_add_match, [gvkey, gvkey, pub_match['hqduns'].item(), pub_match['hqduns'].item(), pub_match['similarity_score'].item(), pub_match['distance'].item(), name_all_match['hqduns'].item(), name_all_match['hqduns'].item(), name_all_match['similarity_score'].item(), name_all_match['distance'].item()])
        except KeyError:
             con.execute(prepared_statement_add_match, [gvkey, gvkey, pub_match['hqduns'].item(), pub_match['hqduns'].item(), pub_match['similarity_score'].item(), pub_match['distance'].item(), name_all_match['hqduns'].item(), name_all_match['hqduns'].item(), name_all_match['similarity_score'].item(), -1])

In [13]:
matching(compustat_df[26000:])
# Call matching with certain gvkey
# matching(compustat_df[compustat_df['gvkey'] == 11058])
# city_comp = compustat_df[compustat_df['gvkey'] == 11058]['city'].iloc[0]
# city_nets = con.execute('SELECT * FROM nets_all WHERE hqduns = 39584008').fetchdf()['hqcity'].iloc[0]
# print(len(city_comp), len(city_nets))

Already processed 9406, SANTAFESNYDERCORP
Already processed 9410, SANYOMANUFACTURINGCORP
Already processed 9411, HILLSHIREBRANDSCO
Already processed 9429, SAVA-STOPINC
Already processed 9433, SAVANNAHFOODS&INDS
Already processed 9435, SAVOYINDINC
Already processed 9462, SCHILLERIND
Already processed 9463, SCHLITZ(JOS.)BREWINGCO
Already processed 9482, SCIENCEMANAGEMENTCORP
Already processed 9491, SCIENTIFICLEASINGINC
Already processed 9496, SCIENTIFICTEXTILESYSTEMS
Already processed 9504, SCOPEIND
Already processed 9505, COLORADOWYOMINGRESERVECO
Already processed 9518, CONSOLIDATEDSYHCORP
Already processed 9533, SEAGALLEYSTORESINC
Already processed 9536, SEAWORLDINC
Already processed 9545, SEAGATE!-OLD
Already processed 9582, SEEQ!INC
Already processed 9610, SERVAMERICAINC
Already processed 9612, SERVICEMERCHANDISECO
Already processed 9615, SERVINATIONALINC/OH
Already processed 9632, SHANLEYCORP
Already processed 9633, SHANNONGRPINC
Already processed 9640, SHATTUCKDENNMININGCORP
Alread

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Processing 3400 of 13782
Processing 3450 of 13782
Processing 3500 of 13782
Processing 3550 of 13782
Processing 3600 of 13782
Processing 3650 of 13782
Processing 3700 of 13782


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Processing 3750 of 13782
Processing 3800 of 13782
Processing 3850 of 13782
Processing 3900 of 13782


...

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Processing 13750 of 13782


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [28]:
con.sql('SELECT * FROM matched').fetchdf().sample(10).sort_values(by='similarity_all', ascending=False)

,gvkey,hqcompany,hqduns_pub,hqcompany_pub,similarity_pub,distance_pub,hqduns_all,hqcompany_all,similarity_all,distance_all
6574,14064,MESTEKINC,4328225,MESTEKINC,1.000000,0.345243,4328225,MESTEKINC,1.000000,0.345243
29093,143910,BRIDGECAPITALHLDGS,791190932,BRIDGECAPITALHLDGS,1.000000,0.000051,791190932,BRIDGECAPITALHLDGS,1.000000,0.000051
28118,39225,LEGALZOOM.COMINC,931244508,LEGALZOOMCOMINC,0.987500,0.003767,931244508,LEGALZOOMCOMINC,0.987500,0.003767
2803,30608,MPSGRPINC,96602008,MPSGRPINC,1.000000,13.151968,86726077,MPGRPINC,0.970370,0.163659
9204,160257,MTONEWIRELESSCORPREDH,-1,None,-1.000000,9999999.000000,830032400,MTONEWIRELESSCORP,0.961905,0.000151
11645,14625,AMERICANPLASTICS&CHEM,-1,None,-1.000000,9999999.000000,621634583,AMERICANPLASTICS&CHEMICALS,0.961538,0.001212
35941,9741,SIMPLEXWIREANDCABLECO,-1,None,-1.000000,9999999.000000,120885335,SIMPLEXWIRE&CABLECO,0.949791,0.017240
27440,26296,GIBRALTARGROWTHCORP,6904429,GIBRALTAR)CORP,0.907172,40.183838,-1,None,-1.000000,9999999.000000
4794,185865,AUSTRALIAACQUISITIONCORP,117943061,AUSTERLITZACQUISITIONCORPII,0.900842,270.568268,-1,None,-1.000000,9999999.000000
23221,61449,GABELLICONV&INCOMESECFD,-1,None,-1.000000,9999999.000000,-1,None,-1.000000,9999999.000000


In [15]:
matched = con.sql('SELECT COUNT(*) FROM matched WHERE similarity_pub >= 0.94').fetchdf()
matched_size = con.sql('SELECT COUNT(*) FROM matched').fetchdf()['count_star()'].iloc[0]
#print percentage of matched companies|
print(matched['count_star()'].iloc[0] / matched_size * 100)
print(matched['count_star()'].iloc[0])

38.73359810969785
15409


In [18]:
# Show statistics
print(f'Number of companies: {matched_size}')
accuracy = con.sql('SELECT COUNT(*) FROM matched WHERE similarity_pub >= 0.95').fetchdf()['count_star()'].iloc[0] / matched_size * 100
accuray_with_all = con.sql('SELECT COUNT(*) FROM matched WHERE similarity_all >= 0.95').fetchdf()['count_star()'].iloc[0] / matched_size * 100 + accuracy
print(f'Accuracy with all companies: {accuray_with_all}%')
print(f'Accuracy only public companies: {accuracy}%')
same_match = pub_match_all / matched_size * 100
print(f'Entries with same match: {same_match}%')
no_pub_percentage = con.sql('SELECT COUNT(*) FROM matched WHERE hqduns_pub = -1').fetchdf()['count_star()'].iloc[0] / matched_size * 100
print(f'Entries with no pub match: {no_pub_percentage}%')
no_pub_but_all_percentage = con.sql('SELECT COUNT(*) FROM matched WHERE hqduns_pub = -1 AND hqduns_all != -1 AND similarity_all >= 0.94').fetchdf()['count_star()'].iloc[0] / matched_size * 100
print(f'Entries with no pub match but all match: {no_pub_but_all_percentage}%')
# All matches, where no pub match and distance < 1
no_pub_but_all_distance_percentage = con.sql('SELECT COUNT(*) FROM matched WHERE hqduns_pub = -1 AND hqduns_all != -1 AND distance_all < 1').fetchdf()['count_star()'].iloc[0] / matched_size * 100
print(f'Entries with no pub match but all match and distance < 1: {no_pub_but_all_distance_percentage}%')

Number of companies: 39782
Accuracy with all companies: 69.92609722990298%
Accuracy only public companies: 36.501432808808005%
Entries with same match: 0.0%
Entries with no pub match: 47.00115630184506%
Entries with no pub match but all match: 7.42546880498718%
Entries with no pub match but all match and distance < 1: 12.354833844452266%


In [17]:
# con.commit()
# con.close()

In [30]:
con.execute('SELECT * FROM matched').fetchdf().to_csv('S:/Users/Hiwi/Maris/name_matching/Data/matched.csv', index=False)
con.execute('SELECT * FROM matched').fetchdf().to_stata('S:/Users/Hiwi/Maris/name_matching/Data/matched.dta')